In [ ]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import torch
from pathlib import Path

from lac.perception.segmentation import Segmentation
from lac.perception.depth import stereo_depth_from_segmentation, project_pixel_to_rover
from lac.control.controller import rock_avoidance_steering, ArcPlanner
from lac.utils.visualization import overlay_mask, overlay_stereo_rock_depths
from lac.utils.plotting import plot_points_rover_frame, plot_path_rover_frame
from lac.utils.frames import invert_transform_mat
from lac.util import load_data, get_positions_from_poses
import lac.params as params

%load_ext autoreload
%autoreload 2

## Obstacle detection


In [ ]:
segmentation = Segmentation()

In [ ]:
# data_path = Path("../../output/NavAgent/map1_preset4_gtnav_steer")
data_path = Path("../../output/DataCollectionAgent/map1_preset9_rocks")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
i = 1154

left_image = Image.open(data_path / "FrontLeft" / f"{i}.png")
right_image = Image.open(data_path / "FrontRight" / f"{i}.png")

left_seg_masks, left_seg_full_mask = segmentation.segment_rocks(left_image.convert("RGB"))
right_seg_masks, right_seg_full_mask = segmentation.segment_rocks(right_image.convert("RGB"))

results = stereo_depth_from_segmentation(
    left_seg_masks, right_seg_masks, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_mask(np.array(left_image), left_seg_full_mask)
left_overlay = overlay_stereo_rock_depths(left_overlay, results)
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(left_overlay)
plt.axis("off")

In [ ]:
x, _, w, _ = cv.boundingRect(results[0]["left_mask"].astype(np.uint8))
width_x = w * results[0]["depth"] / params.FL_X
print(f"Width: {width_x:.2f} m")

In [ ]:
rock_points_rover_frame = []
for rock in results:
    rock_points_rover_frame.append(
        project_pixel_to_rover(rock["left_centroid"], rock["depth"], "FrontLeft", cam_config)
    )
rock_points_rover_frame = np.array(rock_points_rover_frame)
plot_points_rover_frame(rock_points_rover_frame, color="red")

## Dynamics characterization


In [ ]:
data_path = Path("../../output/DataCollectionAgent/map1_preset0_v0.2_w-0.5")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
# Transform poses from world to initial local rover frame
poses_local = [invert_transform_mat(initial_pose) @ pose for pose in poses]

positions_local = get_positions_from_poses(poses_local)
plot_path_rover_frame(positions_local, color="blue")

## Arc path planning


In [ ]:
planner = ArcPlanner()
arcs = planner.candidate_arcs

In [ ]:
fig = plot_points_rover_frame(rock_points_rover_frame, color="red")
# fig = go.Figure()
for arc in arcs:
    fig = plot_path_rover_frame(arc, fig=fig)
fig.show()